In [ ]:
!pip install keybert
!pip install keybert[flair]
!pip install keybert[gensim]
!pip install keybert[spacy]
!pip install keybert[use]
!pip install keybert sentence-transformers flair spacy gensim tensorflow-hub transformers
!python -m spacy download en_core_web_md
!pip install sklearn rouge_score
!pip install datasets
!pip install rouge_score

In [ ]:
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer
from flair.embeddings import TransformerDocumentEmbeddings
import spacy
import gensim.downloader as api
import tensorflow_hub as hub
import flair
import numpy as np
import torch
from transformers import AutoModelForSeq2SeqLM, AutoModel, AutoTokenizer, T5ForConditionalGeneration, BartForConditionalGeneration
from datasets import load_dataset
from rouge_score import rouge_scorer
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
#load Inspec dataset with trust_remote_code=True
ds = load_dataset("taln-ls2n/inspec", trust_remote_code=True)

In [ ]:
texts = [sample["abstract"] for sample in ds["train"]]  #using 'abstract' as the main text
ground_truth_keywords = [sample["keyphrases"] for sample in ds["train"]]  #'keyphrases' contains ground truth


In [ ]:
#load embedding-based models
embedding_models = {
    "sentence-transformers": SentenceTransformer("all-MiniLM-L6-v2"),
    "flair": TransformerDocumentEmbeddings("bert-base-uncased"),
    "roberta": TransformerDocumentEmbeddings("roberta-base"),
    "spacy": spacy.load("en_core_web_md"),
    "gensim": api.load("glove-wiki-gigaword-100"),  # GloVe 100D
    "use": hub.load("https://tfhub.dev/google/universal-sentence-encoder/4"),
}

#categorizing models
bert_models = {
    "bert-keyword-extractor": "yanekyuk/bert-keyword-extractor"
}

generative_models = {
    "vlt5-base-keywords": "Voicelab/vlt5-base-keywords",
    "t5": "t5-small",
    "bart": "facebook/bart-large-cnn",
}

#load tokenizers
tokenizers = {name: AutoTokenizer.from_pretrained(model) for name, model in {**bert_models, **generative_models}.items()}

#load models properly
embedding_models.update({name: AutoModel.from_pretrained(model) for name, model in bert_models.items()})
embedding_models.update({name: AutoModelForSeq2SeqLM.from_pretrained(model) for name, model in generative_models.items()})


In [ ]:
#function to extract embeddings
def get_embedding(model_name, text):
    #Sentence Transformers
    if model_name == "sentence-transformers":
        return embedding_models[model_name].encode([text])[0]

    #Flair or RoBERTa-based models
    elif model_name in ["flair", "roberta"]:
        sentence = flair.data.Sentence(text)
        embedding_models[model_name].embed(sentence)
        return sentence.embedding.cpu().detach().numpy()

    #spaCy embeddings
    elif model_name == "spacy":
        return embedding_models[model_name](text).vector

    #Gensim embeddings like: Word2Vec, GloVe
    elif model_name == "gensim":
        words = text.lower().split()
        vectors = [embedding_models[model_name].get_vector(w) for w in words if w in embedding_models[model_name]]
        return np.mean(vectors, axis=0) if vectors else np.zeros(100)

    #Universal Sentence Encoder (USE)
    elif model_name == "use":
        return embedding_models[model_name]([text])[0].numpy()

    #BERT-based models
    elif model_name in ["bert-keyword-extractor", "vlt5-base-keywords"]:
        tokenizer = tokenizers[model_name]
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
        outputs = embedding_models[model_name](**inputs)
        return outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy()

    #Generative models (T5, BART)
    elif model_name in ["t5", "bart"]:
        tokenizer = tokenizers[model_name]
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
        model_output = embedding_models[model_name](**inputs)
        return model_output.last_hidden_state.mean(dim=1).squeeze().detach().numpy()

    else:
        raise ValueError(f"Unsupported model: {model_name}")


In [ ]:
# run KeyBERT extraction for embedding models
results = {}

#define KeyBERT parameters
keybert_params = {
    "keyphrase_ngram_range": (1, 1),  #single-word keywords
    "top_n": 5,                       #extract top 5 keywords
    "stop_words": "english",          #eemove common words
    "use_mmr": False                  #no diversification
}

for model_name, model in embedding_models.items():
    print(f"Testing model: {model_name}")

    #skip generative models here
    if model_name in generative_models:
        print(f"Skipping {model_name}, as it requires text generation.")
        continue

    kw_model = KeyBERT(model=model)

    extracted_keywords = [
        [kw[0].lower() for kw in kw_model.extract_keywords(text, **keybert_params)]
        for text in texts
    ]

    results[model_name] = extracted_keywords

Testing model: sentence-transformers
Testing model: flair
Testing model: roberta
Testing model: spacy
Testing model: gensim
Testing model: use


Instructions for updating:
Use tf.identity with explicit device placement instead.


Testing model: bert-keyword-extractor


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Testing model: vlt5-base-keywords
Skipping vlt5-base-keywords, as it requires text generation.
Testing model: t5
Skipping t5, as it requires text generation.
Testing model: bart
Skipping bart, as it requires text generation.


In [ ]:
#function for generative keyword extraction
def generate_keywords_with_model(model_name, text):
    tokenizer = tokenizers[model_name]
    model = embedding_models[model_name]

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    #define generation parameters
    generation_params = {
        "max_length": 50,
        "num_beams": 3,
        "temperature": 1.0,  #neutral randomness
        "top_k": 50,
        "top_p": 0.9,
        "repetition_penalty": 1.2,
        "do_sample": True
    }

    with torch.no_grad():
        outputs = model.generate(**inputs, **generation_params)

    return tokenizer.decode(outputs[0], skip_special_tokens=True).lower().split()

In [ ]:
#run keyword generation for generative models
for model_name in generative_models.keys():
    print(f"Generating keywords with {model_name}...")
    extracted_keywords = [generate_keywords_with_model(model_name, text) for text in texts]
    results[model_name] = extracted_keywords

Generating keywords with vlt5-base-keywords...
Generating keywords with t5...
Generating keywords with bart...


/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1427: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (50). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


In [ ]:
#initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

#store ROUGE results
rouge_results = {}

#compute ROUGE scores for each model
for model_name, extracted in results.items():
    print(f"Calculating ROUGE for {model_name}...")

    rouge_scores = {"rouge1": [], "rouge2": [], "rougeL": []}

    for gt, pred in zip(ground_truth_keywords, extracted):
        #convert list of keywords to a single string
        gt_text = " ".join(gt)
        pred_text = " ".join(pred)

        #compute ROUGE scores
        scores = scorer.score(gt_text, pred_text)

        #store results
        rouge_scores["rouge1"].append(scores["rouge1"].fmeasure)
        rouge_scores["rouge2"].append(scores["rouge2"].fmeasure)
        rouge_scores["rougeL"].append(scores["rougeL"].fmeasure)

    #compute average scores
    rouge_results[model_name] = {metric: np.mean(values) for metric, values in rouge_scores.items()}

#print results
for model, scores in rouge_results.items():
    print(f"\nROUGE Scores for {model}:")
    for metric, score in scores.items():
        print(f"  {metric}: {score:.4f}")


Calculating ROUGE for sentence-transformers...
Calculating ROUGE for flair...
Calculating ROUGE for roberta...
Calculating ROUGE for spacy...
Calculating ROUGE for gensim...
Calculating ROUGE for use...
Calculating ROUGE for bert-keyword-extractor...
Calculating ROUGE for vlt5-base-keywords...
Calculating ROUGE for t5...
Calculating ROUGE for bart...

ROUGE Scores for sentence-transformers:
  rouge1: 0.3019
  rouge2: 0.0306
  rougeL: 0.2155

ROUGE Scores for flair:
  rouge1: 0.2077
  rouge2: 0.0101
  rougeL: 0.1499

ROUGE Scores for roberta:
  rouge1: 0.1433
  rouge2: 0.0042
  rougeL: 0.1134

ROUGE Scores for spacy:
  rouge1: 0.1450
  rouge2: 0.0061
  rougeL: 0.1115

ROUGE Scores for gensim:
  rouge1: 0.0709
  rouge2: 0.0017
  rougeL: 0.0587

ROUGE Scores for use:
  rouge1: 0.2513
  rouge2: 0.0195
  rougeL: 0.1861

ROUGE Scores for bert-keyword-extractor:
  rouge1: 0.2723
  rouge2: 0.0269
  rougeL: 0.2017

ROUGE Scores for vlt5-base-keywords:
  rouge1: 0.3860
  rouge2: 0.2245
  rougeL:

In [ ]:
#store cosine similarity results
cosine_similarities = {}

for model_name, extracted in results.items():
    print(f"Calculating Cosine Similarity for {model_name}...")

    cos_sim = []

    for gt, pred in zip(ground_truth_keywords, extracted):
        if not gt or not pred:
            continue

        #get embeddings
        gt_embedding = get_embedding("sentence-transformers", " ".join(gt))
        pred_embedding = get_embedding("sentence-transformers", " ".join(pred))

        #compute cosine similarity
        similarity = cosine_similarity([gt_embedding], [pred_embedding])[0][0]
        cos_sim.append(similarity)

    #compute average similarity
    cosine_similarities[model_name] = np.mean(cos_sim) if cos_sim else 0

#print results
for model, sim in cosine_similarities.items():
    print(f"\nCosine Similarity for {model}: {sim:.4f}")


Calculating Cosine Similarity for sentence-transformers...
Calculating Cosine Similarity for flair...
Calculating Cosine Similarity for roberta...
Calculating Cosine Similarity for spacy...
Calculating Cosine Similarity for gensim...
Calculating Cosine Similarity for use...
Calculating Cosine Similarity for bert-keyword-extractor...
Calculating Cosine Similarity for vlt5-base-keywords...
Calculating Cosine Similarity for t5...
Calculating Cosine Similarity for bart...

Cosine Similarity for sentence-transformers: 0.6394

Cosine Similarity for flair: 0.4551

Cosine Similarity for roberta: 0.3220

Cosine Similarity for spacy: 0.3226

Cosine Similarity for gensim: 0.1457

Cosine Similarity for use: 0.5229

Cosine Similarity for bert-keyword-extractor: 0.5636

Cosine Similarity for vlt5-base-keywords: 0.6390

Cosine Similarity for t5: 0.6181

Cosine Similarity for bart: 0.6950
